The goal of this project is to answer five questions:
- Q1 - What part of my music taste can be described and predicted?
- Q2 - What are my favorite genres?
- Q3 - What are my favorite decades/periods?
- Q4 - What are some albums that I should've liked according to a well-trained model, but didn't?
- Q5 - What are some albums liked by me that a well trained-model expected me to dislike?

All five questions require statistical modelling. Even though on the first glance Q2 and Q3 could be answered only using descriptive statistics, we'll later see how selection bias in the sample obscures causality. In order to reliably estimate impact of genres and decades on my ratings, I need to use many factors at once.

### Libraries and settings

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression, Ridge
from sklearn.metrics import r2_score, mean_squared_error
pd.set_option('display.float_format', lambda x: '%.2f' % x)
np.set_printoptions(suppress=True)
%matplotlib inline

### Loading and preparing data

My ratings are what our models will try to predict. To help us out, we have other users' ratings, as well as some metadata, at our disposal.

In [ ]:
# loading dataframes from data folder
my_ratings = pd.read_csv('my_ratings.csv').set_index('album').rename_axis('album').sort_index()
user_ratings = pd.read_csv('user_ratings.csv').set_index('album').sort_index()
metadata = pd.read_csv('metadata.csv').set_index('album').sort_index()

In [ ]:
my_ratings.head() # 5 example records from my_ratings table

In [ ]:
user_ratings.head() # 5 example records from user_ratings table

In [ ]:
metadata.head() # 5 example records from metadata table

Object `my_ratings` stores the ratings I gave to various albums, EPs and singles. Each album has one rating attached to it and there is no temporal element in the data - we assume all ratings are up to date. My ratings can take value of any real number between 0 and 5.
Other users' ratings are stored in the `user_ratings` object. These ratings' values are between 0.5 and 5.0, with 0.5 increments. `NaN` values mean that a particular user hasn't rated this album. We can expect that appropriate handling of `NaN` values will be very important as we develop the model. Let's see how sparse is the dataset.

In [ ]:
NaN_share = user_ratings.isnull().sum().sum() / np.prod(user_ratings.shape) # share of fields with missing values in the user_ratings table
print(f"{NaN_share:.1%} of the fields are NaNs.")

Other users' ratings comprise mostly albums that I also have rated but there can be a few exceptions. Additionally, my ratings can contain albums that other users haven't rated, or otherwise albums we don't have data on.
Let's ensure that we have the same albums in both datasets, by dropping some rows.

In [ ]:
print(f'Before cleaning - my ratings: {my_ratings.shape[0]}, user ratings: {user_ratings.shape[0]}') # number of records before cleaning

In [ ]:
albums_intersection = list(set(user_ratings.index).intersection(set(my_ratings.index))) # list of albums present in both datasets
my_ratings = my_ratings.loc[albums_intersection] # filter my_ratings
user_ratings = user_ratings.loc[albums_intersection] # filter user_ratings

In [ ]:
print(f'After cleaning - my ratings: {my_ratings.shape[0]}, user ratings: {user_ratings.shape[0]}') # number of records after cleaning

# Exploratory data analysis and feature engineering

Quick check of available data.

In [ ]:
my_vs_avg_rating = my_ratings.join(user_ratings.mean(axis=1).rename('avg_rating')) # real scores and average user ratings in a single dataframe

In [ ]:
ratings_corr = np.corrcoef(my_vs_avg_rating.SCORE, my_vs_avg_rating.avg_rating)[0][1] # correlation coefficient between my ratings and user ratings
print(f'Correlation between my ratings and average user ratings: {ratings_corr:.1%}')

Genres strings can contain multiple genres for the same album. Let's split them so they can be analyzed.

In [ ]:
genres_split = [(index, row.genres.split(', ')) for index, row in metadata.iterrows()] # genres series of strings transformed into series of lists of strings
genres_df = pd.DataFrame(genres_split)
genres_df.set_index(0, inplace = True)
genres_df = genres_df[1].explode().rename('genre') # creating dataframe where one row = one genre tag

What are the most popular genres in my database?

In [ ]:
ax = genres_df.value_counts().head(20).plot(kind='bar', figsize = [12, 6]) # count of genre appearances, top 20
plt.ylabel('count of albums with genre')
ax.bar_label(ax.containers[0]);

How many genres are assigned per album?

In [ ]:
ax = genres_df.reset_index()[0].value_counts().value_counts().sort_index().plot(kind='bar', figsize = [12, 6]) # count of genres per album
plt.xlabel('number of genres per album');
plt.ylabel('count of albums')
ax.bar_label(ax.containers[0]);

What decade were the albums in my database released in?

In [ ]:
(10*np.floor(metadata.year/10)).astype(int).value_counts().sort_index().plot(kind='bar', figsize = [12, 6]) # count of albums per decade
plt.xlabel('release decade');
plt.ylabel('count of albums');

For the purpose of model training let's create binary variables which check the genres strings against a list of words.

In [ ]:
# list containing potentially useful words to check against, based on above lists 
words_to_check = ['Rock', 'Punk', 'Pop', 'Post', 'Noise', 'Experimental', 'Rap', 'Hop', 'Jazz', 'Metal', 'Ambient', 'Indie', 'Art', 'Hardcore', 'Folk', 'Garage', 'Psychedel', 'Songwriter', 'Alternative', 'Industrial', 'Wave', 'Progressive', 'Avant', 'Techno', 'Synth', 'Math', 'Electronic', 'Jangle', 'Drone', 'Hypnagogic', 'Chamber', 'Contemporary', 'Power']

In [ ]:
def check_for_a_word(word):
    """
    Parameters
    __________
    word : str
        Word which we'll check the genre list against.

    Returns
    _______
    check_for_word : function
        Function which can be applied on multiple genre lists.
    """
    def check_for_word(genre_list):
        # Helper function, instance of check_for_a_word, but for a particular word.
        return word in genre_list
    return check_for_word

In [ ]:
for word in words_to_check:
    metadata['Is_'+word] = metadata.genres.apply(check_for_a_word(word)) * 1 # creates multiple new binary features in the metadata dataframe

In [ ]:
metadata = pd.concat([metadata, pd.get_dummies(10*np.floor(metadata.year/10), prefix = 'Is')], axis = 1) # creates multiple binary features related to release decade

## Model training and evaluation functions

In [ ]:
def loocv_regression(
    data, my_ratings, model=Ridge(alpha=100), fill = 2
):
    """
    Perform leave one regression on the provided dataset.

    Parameters
    __________
    data : pd.DataFrame
        Independent variables with which model will be trained.
    my_ratings : pd.DataFrame
        Dependent variable which model attempts to predict.
    model : object
        Model which can be fit on provided data.
    fill : float
        Value to replace NaNs with, for user ratings.

    Returns
    _______
    result : pd.DataFrame
        Predictions for each observation.
    """
    data_f = data.fillna(fill)
    i = 1
    albums = []
    predictions = []
    for index, _ in my_ratings.iterrows():
        i += 1
        X = data_f.loc[my_ratings.index].drop(index)
        y = my_ratings.drop(index)["SCORE"]
        model.fit(X, y)
        row_df = data_f.loc[[index]]
        pred = model.predict(row_df)
        albums.append(index)
        predictions.append(pred[0])
        print(f"Predicted rating for {index}: {pred[0]:.2f}")
    result = pd.DataFrame({"album": albums, "prediction": predictions}).sort_values(
        "prediction", ascending=False
    ).set_index('album')
    return result


In [ ]:
def evaluate(predictions, my_ratings):
    """
    Evaluates predictions of my ratings generated by a model.
    Parameters
    __________
    predictions : pd.DataFrame
        Predictions generated by a model.
    my_ratings : pd.DataFrame
        Dependent variable which the model tried to predict.

    Returns
    _______
    r2 : float
        Coefficient of determination.
    rmse : float
        Root mean squared error.
    """
    df = pd.concat([predictions, my_ratings], axis = 1)
    r2 = r2_score(df.SCORE, df.prediction)
    rmse = mean_squared_error(df.SCORE, df.prediction, squared = False)
    return r2, rmse

## Model A

In [ ]:
result_A = loocv_regression(user_ratings, my_ratings, verbose = False) # run leave-one-out regression
print(evaluate(result_A, my_ratings)) # calculate prediction error
score_pred_A = pd.concat([result_A, my_ratings], axis = 1) # real scores and predictions in one table

In [ ]:
sns.scatterplot(x = 'SCORE', y = 'prediction', data = score_pred_A, alpha=0.25) # visualization of results

## Model B

In [ ]:
result_B = loocv_regression(metadata.drop(columns=['genres', 'year']), my_ratings, verbose = False) # run leave-one-out regression
print(evaluate(result_B, my_ratings)) # calculate prediction error

In [ ]:
result_B.prediction.round(1).value_counts().sort_index().plot(kind='bar') # visualization of prediction spread 

## Model C

In [ ]:
full_data = pd.concat([user_ratings, metadata.drop(columns = ['ID', 'genres', 'year']).loc[user_ratings.index]], axis = 1) # user_ratings and metadata joined into one table

In [ ]:
result_C = loocv_regression(full_data, my_ratings, verbose = False) # run leave-one-out regression
print(evaluate(result_C, my_ratings)) # calculate prediction error
score_pred_C = pd.concat([result_C, my_ratings], axis = 1) # real scores and predictions in one table

In [ ]:
sns.scatterplot(x = 'SCORE', y = 'prediction', data = score_pred_C, alpha=0.25) # visualization of results

### Answer to Question 1: What part of my music taste can be described and predicted?
Model C had a final R-squared coefficient of 20.4%, and that's the part of variance which could be explained by the model. This value can probably be higher with additional features or different models, but we have to keep in mind that we probably won't be able to predict most of the variance, since the topic we're tackling is subjective opinions.

## Inference

In order to look at coefficients, we'll run the model on all available data.

In [ ]:
model = Ridge(alpha = 100) # initialization of new model
model.fit(full_data.fillna(2), my_ratings.SCORE) # single fit for all albums, without cross-validation
coefs = pd.DataFrame({'feature': full_data.columns, 'coef': model.coef_}) # dataframe with variable coefficients

How were the coefficients distributed for user ratings?

In [ ]:
coefs.loc[~coefs.feature.str.startswith('Is')].coef.hist() # only user-related coefficients
plt.xlabel('coefficient');
plt.ylabel('count of users');

How do coefficients look like for genre variables?

In [ ]:
coefs.loc[coefs.feature.str.startswith('Is_')].loc[(~coefs.feature.str.contains('19')) & (~coefs.feature.str.contains('20'))].sort_values('coef').set_index(['feature']).coef.plot(kind='bar') # only genre-related coefficients
plt.xlabel('genre');
plt.ylabel('coefficient');

### Answer to Question 2: What are my favorite genres?
Phrases which increase predicted ratings are Punk (like Post-Punk or Hardcore Punk), Hop (like Hip Hop, Glitch Hop etc), Hypnagogic (like Hypnagogic Pop), and Indie (like Indie Rock or Indie Surf).
Phrases for which score predictions are lower, are Industrial, Electronic, Math (like Math Rock), Pop and Jazz.

How do coefficients look like for decade variables?

In [ ]:
coefs.loc[coefs.feature.str.startswith('Is_')].loc[(coefs.feature.str.contains('19')) | (coefs.feature.str.contains('20'))].sort_values('feature').set_index(['feature']).coef.plot(kind='bar') # only decade-related coefficients
plt.xlabel('decade');
plt.ylabel('coefficient');

### Answer to Question 3: What are my favorite decades/periods?
Albums from 2010s and especially 2020s seem to be highly rated by me. On the other hand, albums from 1970s and 1990s especially, received lower ratings. This already takes into account other users' ratings, as well as genres, so it's not a simple comparison between scores per decade, but an attempt to measure how decade influences predicted score.

Let's finally use last LOO CV predictions to see which albums were most over- and underrated by the model.

In [ ]:
score_pred_C['diff'] = score_pred_C.SCORE - score_pred_C.prediction # prediction error

In [ ]:
score_pred_C.sort_values('diff').head() # negative prediction error

### Answer to Question 3:  What are some albums that I should've liked according to a well-trained model, but didn't?
List is provided above. Maybe I should listen to these albums again?

In [ ]:
score_pred_C.sort_values('diff', ascending = False).head() # positive prediction error

### Answer to Question 5: What are some albums liked by me that a well trained-model expected me to dislike?
List is provided above. It could be a good idea to check why my model stumbles on those. Maybe there are some additional variables which can address this and improve performance of the model on these outliers.

## Conclusion

Even with a simple model it's possible to explain 20% of the variance in my ratings. Some variables required feature engineering, while others were ready to use. Linear model used (ridge regression) probably didn't do as well as more complicated models would, but it offered interpretability of coefficients, which was very important to answer some of the questions in the analysis. [Link to Medium article describing results in depth](https://medium.com/@mariusz.sz/why-do-i-like-music-1ef8477b469f)